In [ ]:
import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf
import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics

from LPdata import read_label_data
from MLP import MLP


In [ ]:
# Load data for neuron network
dataFileName = 'raw.csv'
dataFilePath = pathlib.Path.cwd().joinpath('demo_data', f'{dataFileName}')
segFileName = 'seg_point.csv'
segFilePath = pathlib.Path.cwd().joinpath('demo_data', f'{segFileName}')
# Extract data and get label of each row
data = read_label_data(dataFilePath, segFilePath)

In [ ]:
def normalise(n, maxNum, minNum):
    return (n - minNum) / (maxNum - minNum)
# Normalise data
for col in range(data.shape[1]):
    if col != (data.shape[1] - 1):
        maxNum = np.max(data.iloc[:, col])
        minNum = np.min(data.iloc[:, col])
        data.iloc[:, col] = data.iloc[:, col].apply(normalise, args=(maxNum, minNum,))

In [ ]:
# Randomise data
data = data.sample(frac=1)
# Split data into training set and testing set
train, test = train_test_split(data, test_size=0.1)
trainData, trainLabel = train.drop(columns='LABEL').values, train['LABEL'].values
testData, testLabel = test.drop(columns='LABEL').values, test['LABEL'].values

In [ ]:
# Hyper-parameters
EPOCHS = 50
BATCH_SIZE = 400
# Build MLP
model = MLP()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(trainData, trainLabel, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

In [ ]:
# plot using ggplot style
plt.style.use('ggplot')
# plot loss of MLP during training step
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

In [ ]:
# Test model
testPred = model.predict(testData)
# Take the class with the highest probability from the test predictions
testPredMax = np.argmax(testPred, axis=1)
LABELS = ['NONE', 'SLR-R', 'SLR-L', 'SAE-R', 'SAE-L', 'KE-R', 'KE-L']
# Show confusion matrix
matrix = metrics.confusion_matrix(testLabel, testPredMax)
plt.figure(figsize=(6, 4))
sns.heatmap(matrix,
            cmap='coolwarm',
            linecolor='white',
            linewidths=1,
            xticklabels=LABELS,
            yticklabels=LABELS,
            annot=True,
            fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()